In [150]:
import pandas as pd

# load the file
pymoli = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
pymoli_df = pd.read_csv(pymoli).dropna()
pymoli_df.head().style.hide_index()

Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.530000
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.560000
2,Ithergue48,24,Male,92,Final Critic,4.880000
3,Chamassasya86,24,Male,100,Blindscythe,3.270000
4,Iskosia90,23,Male,131,Fury,1.440000


In [151]:
# Create player demographics dataframe for later use
demographics = pymoli_df[["Gender", "SN", "Age"]].drop_duplicates()

# Create player_count variable for later use
player_count = pymoli_df["SN"].nunique()

total_players_df = pd.DataFrame({"Total Players": [player_count]}).style.hide_index()
total_players_df

Total Players
576


In [152]:
#unique values core each category
#note there are 780 purchase IDs but only 576 unique screen names(i.e. unique people that bought items)
pymoli_df.nunique()


Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        179
Item Name      179
Price          145
dtype: int64

In [153]:
#Purchasing Analysis (Total)
unique_items = pymoli_df["Item ID"].nunique()
average_price = pymoli_df["Price"].mean()
number_purchases = pymoli_df["Purchase ID"].nunique()
total_revenue = pymoli_df["Price"].sum()


Values = pd.DataFrame({"Number of Unique Items": [unique_items], 
                       "Average Price": [average_price], 
                       "Number of Purchases": [number_purchases], 
                       "Total Revenue": [total_revenue]})

Values["Average Price"] = Values["Average Price"].map('${:,.2f}'.format)
Values["Total Revenue"] = Values["Total Revenue"].map('${:,.2f}'.format)

Values

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [154]:
#Gender data frame, gives you purchase count by gender
#this data frame has no duplicates 
gender = demographics["Gender"].value_counts()
gender

Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [155]:
#Gender Demographics
percent_male_players = (484 / 576)*100
count_male_players = 484
percent_female_players = (81 / 576)*100
count_female_players = 81
percent_other_players = (11 / 576)*100
count_other_players = 11

#create gender values data frame
gender_values = pd.DataFrame(columns = ["Total Players", "Percentage of Players"], 
                             index = ["Male", "Female", "Other / Non-Disclosed"])

gender_values.loc["Male"] = [count_male_players, percent_male_players]
gender_values.loc["Female"] = [count_female_players, percent_female_players]
gender_values.loc["Other / Non-Disclosed"] = [count_other_players, percent_other_players]

gender_values["Percentage of Players"] = gender_values["Percentage of Players"].map('{:,.2f}%'.format)

gender_values

,Total Players,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [156]:
#Gender purchase count data frame, gives you purchase count by gender including duplicates
gender_count = pymoli_df["Gender"].value_counts()
gender_count

Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64

In [157]:
#groupby and perform calculations
gender_purchase_total = pymoli_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")

gender_avg_price = pymoli_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")

In [158]:
#calculations for summary table
purchase_count_male = 652
purchase_count_female = 113
purchase_count_other = 15
avg_price_male = gender_avg_price["Male"]
male_total_value = gender_purchase_total["Male"]
male_avg_total = male_total_value / count_male_players
avg_price_female = gender_avg_price["Female"]
female_total_value = gender_purchase_total["Female"]
female_avg_total = female_total_value / count_female_players
avg_price_other = gender_avg_price["Other / Non-Disclosed"]
other_total_value = gender_purchase_total["Other / Non-Disclosed"]
other_avg_total = other_total_value / count_other_players


In [163]:
# Purchasing Analysis (Gender)

purchase_summary = pd.DataFrame(columns = ["Purchase Count", "Avg Purchase Price", 
                                           "Total Purchase Value", "Avg Total Purchase per Person"], 
                             index = ["Male", "Female", "Other / Non-Disclosed"])

purchase_summary.loc["Male"] = [purchase_count_male, avg_price_male, male_total_value, male_avg_total]
purchase_summary.loc["Female"] = [purchase_count_female, avg_price_female, female_total_value, female_avg_total]
purchase_summary.loc["Other / Non-Disclosed"] = [purchase_count_other, avg_price_other, 
                                                 other_total_value, other_avg_total]

#format and map
purchase_summary["Avg Purchase Price"] = purchase_summary["Avg Purchase Price"].map('${:,.2f}'.format)
purchase_summary["Total Purchase Value"] = purchase_summary["Total Purchase Value"].map('${:,.2f}'.format)
purchase_summary["Avg Total Purchase per Person"] = purchase_summary["Avg Total Purchase per Person"].map('${:,.2f}'.format)

purchase_summary

,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [204]:
#Age Demographics

# Create bins for ages
bins = [0, 10, 15, 20, 25, 30, 35, 40, 125]

# Create labels for these bins
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"]
    
    
# Slice the data and place it into bins
pd.cut(pymoli_df["Age"], bins, labels=group_labels).head()

0    15-19
1    35-39
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [205]:
pymoli_df["Age Group"] = pd.cut(pymoli_df["Age"], bins, labels=group_labels)
pymoli_df.head().style.hide_index()

Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.530000,15-19
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.560000,35-39
2,Ithergue48,24,Male,92,Final Critic,4.880000,20-24
3,Chamassasya86,24,Male,100,Blindscythe,3.270000,20-24
4,Iskosia90,23,Male,131,Fury,1.440000,20-24


In [227]:
# Create a GroupBy object based upon "group" and count number in each age group
age_total_count = pymoli_df.groupby(["Age Group"]).count()["Age"].rename("Total Count")
g_avg_price = pymoli_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")

Age Group
<10       32
10-14     54
15-19    200
20-24    325
25-29     77
30-34     52
35-39     33
40+        7
Name: Purchase Count, dtype: int64

In [225]:
#DEBUG, NUMBERS ARE NOT CORRECT


In [232]:
#Calculate Age Demographics totals and percentages:
under_ten_total = age_total_count["<10"]
age10to14_total = age_total_count["10-14"]
age15to19_total = age_total_count["15-19"]
age20to24_total = age_total_count["20-24"]
age25to29_total = age_total_count["25-29"]
age30to34_total = age_total_count["30-34"]
age35to39_total = age_total_count["35-39"]
age40_older_total = age_total_count["40+"]

percent_under_ten = (32 / 576)*100
percent_10to14 = (54 / 576)*100
percent_15to19 = (200 / 576)*100
percent_20to24 = (325 / 576)*100
percent_25to29 = (77 / 576)*100
percent_30to34 = (52 / 576)*100
percent_35to39 = (33 / 576)*100
percent_40_older = (7 / 576)*100

#create age demographic data frame
age_demographics = pd.DataFrame(columns = ["Total Count", "Percentage of Players"], 
                             index = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"])

age_demographics.loc["<10"] = [under_ten_total, percent_under_ten]
age_demographics.loc["10-14"] = [age10to14_total, percent_10to14]
age_demographics.loc["15-19"] = [age15to19_total, percent_15to19]
age_demographics.loc["20-24"] = [age20to24_total, percent_20to24]
age_demographics.loc["25-29"] = [age25to29_total, percent_25to29]
age_demographics.loc["30-34"] = [age30to34_total, percent_30to34]
age_demographics.loc["35-39"] = [age35to39_total, percent_35to39]
age_demographics.loc["40+"] = [age40_older_total, percent_40_older]

#format and map
age_demographics["Percentage of Players"] = age_demographics["Percentage of Players"].map('{:,.2f}%'.format)

age_demographics

,Total Count,Percentage of Players
<10,32,5.56%
10-14,54,9.38%
15-19,200,34.72%
20-24,325,56.42%
25-29,77,13.37%
30-34,52,9.03%
35-39,33,5.73%
40+,7,1.22%


In [218]:
#Purchasing Analysis by Age:

# Create bins for ages
bins = [0, 10, 15, 20, 25, 30, 35, 40, 125]

# Create labels for these bins
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"]
    
    
# Slice the data and place it into bins
pd.cut(pymoli_df["Age"], bins, labels=group_labels).head()

0    15-19
1    35-39
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [219]:
pymoli_df["Age Group"] = pd.cut(pymoli_df["Age"], bins, labels=group_labels)
pymoli_df.head().style.hide_index()

Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.530000,15-19
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.560000,35-39
2,Ithergue48,24,Male,92,Final Critic,4.880000,20-24
3,Chamassasya86,24,Male,100,Blindscythe,3.270000,20-24
4,Iskosia90,23,Male,131,Fury,1.440000,20-24


In [231]:
# GroupBy object based upon "age group" and perform calculations
age_purchase_count = pymoli_df.groupby(["Age Group"]).count()["Age"].rename("Purchase Count")
age_avg_price = pymoli_df.groupby(["Age Group"]).mean()["Price"].rename("Average Purchase Price")
age_total_purchase = pymoli_df.groupby(["Age Group"]).sum()["Price"].rename("Total Purchase Value")

age_purchase_count

Age Group
<10       32
10-14     54
15-19    200
20-24    325
25-29     77
30-34     52
35-39     33
40+        7
Name: Purchase Count, dtype: int64

In [241]:
#Calculate Age Analysis:
purchase_count_under_ten = 32
purchase_count_age10to14 = 54
purchase_count_age15to19 = 200
purchase_count_age20to24 = 325
purchase_count_age25to29 = 77
purchase_count_age30to34 = 52
purchase_count_age35to39 = 33
purchase_count_age40_older = 7


avg_price_under_ten = age_avg_price["<10"]
avg_price_age10to14 = age_avg_price["10-14"]
avg_price_age15to19 = age_avg_price["15-19"]
avg_price_age20to24 = age_avg_price["20-24"]
avg_price_age25to29 = age_avg_price["25-29"]
avg_price_age30to34 = age_avg_price["30-34"]
avg_price_age35to39 = age_avg_price["35-39"]
avg_price_age40_older = age_avg_price["40+"]

age_total_purchase_under_ten = age_total_purchase["<10"]
age_total_purchase_age10to14 = age_total_purchase["10-14"]
age_total_purchase_age15to19 = age_total_purchase["15-19"]
age_total_purchase_age20to24 = age_total_purchase["20-24"]
age_total_purchase_age25to29 = age_total_purchase["25-29"]
age_total_purchase_age30to34 = age_total_purchase["30-34"]
age_total_purchase_age35to39 = age_total_purchase["35-39"]
age_total_purchase_age40_older = age_total_purchase["40+"]

avg_total_under_ten = age_total_purchase_under_ten / purchase_count_under_ten
avg_total_age10to14 = age_total_purchase_age10to14 / purchase_count_age10to14
avg_total_age15to19 = age_total_purchase_age15to19 / purchase_count_age15to19
avg_total_age20to24 = age_total_purchase_age20to24 / purchase_count_age20to24
avg_total_age25to29 = age_total_purchase_age25to29 / purchase_count_age25to29
avg_total_age30to34 = age_total_purchase_age30to34 / purchase_count_age30to34
avg_total_age35to39 = age_total_purchase_age35to39 / purchase_count_age35to39
avg_total_age40_older = age_total_purchase_age40_older / purchase_count_age40_older

#create age analysis data frame
age_analysis = pd.DataFrame(columns = ["Purchase Count", "Avg Purchase Price", "Total Purchase Value",
                                          "Avg Total Purchase per Person"], 
                             index = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"])

age_analysis.loc["<10"] = [purchase_count_under_ten, avg_price_under_ten, age_total_purchase_under_ten, avg_total_under_ten]
age_analysis.loc["10-14"] = [purchase_count_age10to14, avg_price_age10to14, age_total_purchase_age10to14, avg_total_age10to14]
age_analysis.loc["15-19"] = [purchase_count_age15to19, avg_price_age15to19, age_total_purchase_age15to19, avg_total_age15to19]
age_analysis.loc["20-24"] = [purchase_count_age20to24, avg_price_age20to24, age_total_purchase_age20to24, avg_total_age20to24]
age_analysis.loc["25-29"] = [purchase_count_age25to29, avg_price_age25to29, age_total_purchase_age25to29, avg_total_age25to29]
age_analysis.loc["30-34"] = [purchase_count_age30to34, avg_price_age30to34, age_total_purchase_age30to34, avg_total_age30to34]
age_analysis.loc["35-39"] = [purchase_count_age35to39, avg_price_age35to39, age_total_purchase_age35to39, avg_total_age35to39]
age_analysis.loc["40+"] = [purchase_count_age40_older, avg_price_age40_older, age_total_purchase_age40_older, avg_total_age40_older]

#format and map
age_analysis["Avg Purchase Price"] = age_analysis["Avg Purchase Price"].map('${:,.2f}'.format)
age_analysis["Total Purchase Value"] = age_analysis["Total Purchase Value"].map('${:,.2f}'.format)
age_analysis["Avg Total Purchase per Person"] = age_analysis["Avg Total Purchase per Person"].map('${:,.2f}'.format)

age_analysis


,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,32,$3.40,$108.96,$3.40
10-14,54,$2.90,$156.60,$2.90
15-19,200,$3.11,$621.56,$3.11
20-24,325,$3.02,$981.64,$3.02
25-29,77,$2.88,$221.42,$2.88
30-34,52,$2.99,$155.71,$2.99
35-39,33,$3.40,$112.35,$3.40
40+,7,$3.08,$21.53,$3.08


In [ ]:
a